In [21]:
from __future__ import print_function

import sys
import numpy as np
from time import time
import matplotlib.pyplot as plt 
import math

sys.path.append('/home/xilinx')
from pynq import Overlay
from pynq import allocate

if __name__ == "__main__":
    print("Entry:", sys.argv[0])
    print("System argument(s):", len(sys.argv))

    print("Start of \"" + sys.argv[0] + "\"")

    ol = Overlay("frft.bit")
    ipFRFT = ol.frft_0

    fi_indat = open("input_data.txt", "r+")
    fi_ref = open("ref_result.txt", "r+")
    fi_inang = open("input_angle.txt", "r+")
    
    numSamples = 256

    inBufferi = allocate(shape=(numSamples,), dtype=np.int32)
    inBufferq = allocate(shape=(numSamples,), dtype=np.int32)    
    outBufferi = allocate(shape=(numSamples,), dtype=np.int32)
    outBufferq = allocate(shape=(numSamples,), dtype=np.int32)
    outrefi = allocate(shape=(numSamples,), dtype=np.int32)
    outrefq = allocate(shape=(numSamples,), dtype=np.int32)
    
    
    line = fi_indat.readline()
    indat = line.split()
    line = fi_ref.readline()
    inref = line.split()
    line = fi_inang.readline()
    inang = line.split()
    ang = int(float(inang[0])*(2**14))
    print(ang/(2**14))
    
    
    for i in range(256):
        inBufferi[i] = int(float(indat[i*2])*(2**18))
        inBufferq[i] = int(float(indat[i*2+1])*(2**18))
        #print("in = "+str(float(inBufferi[i])/(2**18))+"+i*"+str(float(inBufferq[i])/(2**18)))
        outrefi[i] = int(float(inref[i*2])*(2**18))
        outrefq[i] = int(float(inref[i*2+1])*(2**18))
    fi_indat.close()
    fi_ref.close()
    fi_inang.close()

    
    timeKernelStart = time()
    ipFRFT.write(0x30, ang)
    ipFRFT.write(0x20, inBufferi.device_address)
    ipFRFT.write(0x28, inBufferq.device_address)
    ipFRFT.write(0x10, outBufferi.device_address)
    ipFRFT.write(0x18, outBufferq.device_address)
    ipFRFT.write(0x00, 0x01)
    while (ipFRFT.read(0x00) & 0x4) == 0x0:
        continue
    timeKernelEnd = time()
    print("Kernel execution time: " + str(timeKernelEnd - timeKernelStart) + " s")
    
    err_acc = 0;
    for i in range(256):
        print("outref = "+str(float(outrefi[i])/(2**18))+"+i*"+str(float(outrefq[i])/(2**18))+",   output = "+str(float(outBufferi[i])/(2**18))+"+i*"+str(float(outBufferq[i])/(2**18)))
        err_acc = err_acc+ float(outrefi[i]-outBufferi[i])/(2**18) *  float(outrefi[i]-outBufferi[i])/(2**18)
        err_acc = err_acc+ float(outrefq[i]-outBufferq[i])/(2**18) *  float(outrefq[i]-outBufferq[i])/(2**18)
        
    err_acc = math.sqrt(err_acc/256)
    print("err = "+ str(err_acc))

    print("============================")
    print("Exit process")



Entry: /usr/lib/python3/dist-packages/ipykernel_launcher.py
System argument(s): 3
Start of "/usr/lib/python3/dist-packages/ipykernel_launcher.py"
-2.650390625
Kernel execution time: 0.005889892578125 s
outref = 0.5685615539550781+i*0.4053764343261719,   output = 0.5649299621582031+i*0.40576934814453125
outref = -0.20750045776367188+i*-0.6497459411621094,   output = -0.207855224609375+i*-0.6512298583984375
outref = 0.0308074951171875+i*0.6279754638671875,   output = 0.028560638427734375+i*0.6300849914550781
outref = 0.28990936279296875+i*-0.62255859375,   output = 0.2963981628417969+i*-0.6189804077148438
outref = -0.4095802307128906+i*0.40593719482421875,   output = -0.411651611328125+i*0.40038299560546875
outref = 0.5374336242675781+i*-0.2812042236328125,   output = 0.5321769714355469+i*-0.28216552734375
outref = -0.5183906555175781+i*0.050830841064453125,   output = -0.5180244445800781+i*0.041683197021484375
outref = 0.4892578125+i*0.036579132080078125,   output = 0.4871025085449219+i